<a id='Top'></a>

# Importing the Dataset


                                                   By: Jun Ho Lee
___


<a id='Table of Contents'></a>
## Table of Contents

1. <a href='#Introduction'>Introduction</a>
2. <a href='#Import Dataset'>Importing the Dataset</a>
3. <a href='#Memory Reduction'>Memory Reduction</a>
4. <a href='#JSON Format'>JSON Format</a>

<a href='#Top'>Back to Top</a>

<a id='Introduction'></a>
***


### 1. Introduction 

The dataset I will be using comes from **United States Department of Labor on Foreign Labor Certification.** [Dataset Link](https://www.foreignlaborcert.doleta.gov/performancedata.cfm)  

The dataset is from Fiscal Year 2018 with a reporting period from October 1, 2017 through September 30, 2018. To access the dataset from the URL provided above, click on the Disclosure Data tab and download the ‘H-1B_FY2018.xlsx’ dataset under LCA Programs (H-1B, H-1B1, E-3). The dataset dictionary is downloadable under the File Structure column. (see blue box)

![](imgs/data_boxed_small.png)


<a href='#Top'>Back to Top</a>

<a id='Import Dataset'></a>
___

### 2. Importing the Dataset

The original downloaded file is in .xlsx extension. Pandas do [support](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html) both .xls and .xlsx file extensions, but the performance is notoriously [slow](https://stackoverflow.com/questions/28766133/faster-way-to-read-excel-files-to-pandas-dataframe)! Reading in CSV format is much faster and luckily, we can save the excel file into a csv format. Thus, we will use pandas.read_csv() to read in the converted CSV format. 

* **Don't actually read in the csv file! (Reason explained <a href='#Reason'>below</a>)**

**A. Import necessry libraries**

In [1]:
import pandas as pd

**B. Read in csv file**

In [2]:
# h1b_df = pd.read_csv("data/H-1B_Disclosure_Data_FY2018_EOY.csv", low_memory=False)
# h1b_df.info(memory_usage='deep')

<a id='Reason'></a>
**C. Reason**  

Even though .read_csv() is relatively faster than .read_excel(), this operation will eat up your RAM and slow down your computer performance. If we actually check the memory usage for loading in this dataset, it is actually around ~1.6GB, which can be quite costly, especially for users with low RAM. This particular dataset was only originally around ~200MB but if you imagine blindly running .read_csv() on a larger dataset, it might crash your computer. To prevent these issues, it is essential to reduce memory usage when reading in a particularly large dataset. 


Extensive RAM usage occurs because python by default reads in columns using the highest bytesize `dtype` available (`float64`, `int64` etc.) `String` type is the most costly, since the length of the `String` is proportional to its memory usage. We can solve this problem by downcasting `float64` and `int64` and converting `String` into categoricals. 

<a href='#Top'>Back to Top</a>


<a id='Memory Reduction'></a>
***

### 3. Memory Reduction

- **Necessity:** Prevent potential computer crashes when reading in large datasets. 


- **Objective:** Reduce memory usage by downcasting numerical values and converting string objects into categorical values whenever feasible. Use the converted datatypes as parameters for reading in the dataset.  


- **Method:** 
    1. Downcast `int64` 
    2. Downcast `float64` 
    3. Convert `object` (strings) to `categoricals`


- **Reference:** [Dataquest](https://www.dataquest.io/blog/pandas-big-data/)

***A. Before Getting Started:*** Make a Custom Function to check Memory Usage 
- Will be useful to determine the memory usage of the final optimized dataset! 

In [3]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

**B. Read in first n lines of csv file to determine dtypes**

In [4]:
n = 10000  # keep n high to capture as many unique values as possible
df = pd.read_csv("data/H-1B_Disclosure_Data_FY2018_EOY.csv", nrows=n)

# # Drop columns of mixed type 
drop_columns = ['EMPLOYER_PHONE','EMPLOYER_PHONE_EXT']
df.drop(drop_columns, axis=1, inplace=True)


**C. Downcasting numerical data types (ints / floats)**

In [5]:
# # downcast will only work if the final downcasted version uses less memory than original

# # downcasting ints
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric, downcast='unsigned')

# # downcasting floats
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric, downcast='float') 


**D. Converting object data types to Categoricals**
- Note: Blind conversion of `objects` to `Categoricals` is not preferred. If the number of unique values is close to the total number of values in a column, `Categorical` type will end up using more memory! Thus it is important to initially explore the data and make a judgement call based on the number of unique values in a column. Here, I used a cutoff threshold of 50%, meaning I converted those columns where less than 50% of the values were unique values into `Categoricals`. 
- Note2: Normal string methods don't work on `Categoricals`. 
- Note3: Columns that represent dates should be converted to `datetime` for easier `datetime` operation / calculation.

In [6]:
# # Date columns to drop. Need to retain them as object dtype 
# # (later use parse_dates WHILE reading in the dataset)
date_columns = ['CASE_SUBMITTED','DECISION_DATE','EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE','ORIGINAL_CERT_DATE']

# # Select columns with 'object' dtype
df_obj = df.select_dtypes(include=['object'])

# # Drop the 'Date' Columns since they'll have to be in datetime dtype later
# # Otherwise, 'Date' Columns will get converted to 'Categorical'
df_obj_wo_dates = df_obj.drop(date_columns, axis=1)


**E. Only convert columns where less than 50% are unique values**

In [7]:
# Only convert columns where less than 50% are unique values

converted_obj_wo_dates = pd.DataFrame()
for col in df_obj_wo_dates.columns:
    num_unique_values = len(df_obj_wo_dates[col].unique())
    num_total_values = len(df_obj_wo_dates[col])
    if num_unique_values / num_total_values < 0.5:  # threshold = 50%
        converted_obj_wo_dates.loc[:,col] = df_obj_wo_dates[col].astype('category')
    else:
        converted_obj_wo_dates.loc[:,col] = df_obj_wo_dates[col]

**F. Concatenate the converted dataframes into final Dataframe**
- Final Dataframe will have dtypes of our interest (Downcasted and `Categorical`)


In [8]:
# # Concatenate all the converted dataframes and evaluate memory usage
optimized_sample = pd.concat([converted_int, converted_float, converted_obj_wo_dates], axis=1) 


**G. Parse out the datatypes from optimized_df** 

In [9]:
# # Get the dtypes from the optimized_sample dataframe
dtypes = optimized_sample.dtypes

col_name = dtypes.index                      # name of the column 
col_types = [i.name for i in dtypes.values]  # datatype of the column

# # Save column name and types into a dictionary 
# # {name: type}
column_types = dict(zip(col_name, col_types))


*Function of the code above*

In [10]:
def get_col_dtypes(df):
    dtypes = df.dtypes

    col_name = dtypes.index                      # name of the column 
    col_types = [i.name for i in dtypes.values]  # datatype of the column

    # # Save column name and types into a dictionary 
    # # {name: type}
    column_types = dict(zip(col_name, col_types))
    return column_types

**H. Read in the dataset with our optimized data type parameters!** 

In [11]:
final_cols = df.columns
 
optimized_full = pd.read_csv("data/H-1B_Disclosure_Data_FY2018_EOY.csv", usecols=final_cols, dtype=column_types, low_memory=False, parse_dates=date_columns,infer_datetime_format=True)
print(mem_usage(optimized_full))

204.63 MB


**I. Conclusion**

As you can see, the memory usage of the new optimized dataset is only 204MB! This is a 800% increase in performance when compared to initially blindly reading in the dataset. (~1.6GB used)

<a href='#Top'>Back to Top</a>

<a id='JSON Format'></a>
___


### 4. Save Parameters to JSON Format

- **Objective:** Save the parameters used for reading in the optimized dataframe into a JSON file format so that other jupyter notebooks may read in the same dataset efficiently with the same parameters.  



**A. Make a dictionary using the original parameters**

In [12]:
column_types  # type (dict)
date_columns  # type (list)
final_columns = list(df.columns) # type (list) # df.columns type=(Index) -> must convert to list 

t_list = [column_types, date_columns, final_columns]
t_string = ['c_type','c_date','c_final']

# Make a dictionary 
params_dict = {}

for i in range(len(t_list)):
    name = t_string[i]
    params_dict[name] = t_list[i]

**B. Convert dictionary to JSON file**

In [13]:
# convert to json format
import json 

with open('data/h1b_df_params.json', 'w') as fp:
    json.dump(params_dict, fp)  # convert dict as a json file format! 

**C. Conclusion**

- Parameters have now been saved to a JSON file in the current working directory so that other jupyter noteboks can easily access the optimized column types when reading in this particular dataset.

<a href='#Top'>Back to Top</a>

#### END OF NOTEBOOK
<a href='#Top'>Back to Top</a>